In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandasql as psql # for SQL

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

database_path = "/kaggle/input/videogamesales/vgsales.csv"

/kaggle/input/videogamesales/vgsales.csv


In [2]:
# Python
data = pd.read_csv("/kaggle/input/videogamesales/vgsales.csv")
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
data.info()

#Dropping N/A Years
data = data.dropna(subset=['Year'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [4]:
data['Platform'].value_counts() #Plataform

Platform
DS      2133
PS2     2127
PS3     1304
Wii     1290
X360    1235
PSP     1197
PS      1189
PC       943
GBA      811
XB       803
GC       542
3DS      500
PSV      412
PS4      336
N64      316
SNES     239
XOne     213
SAT      173
WiiU     143
2600     116
NES       98
GB        97
DC        52
GEN       27
NG        12
SCD        6
WS         6
3DO        3
TG16       2
GG         1
PCFX       1
Name: count, dtype: int64

In [5]:
data.Genre.unique() #Genre

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

In [6]:
data['Publisher'].value_counts() #Publisher

Publisher
Electronic Arts                 1339
Activision                       966
Namco Bandai Games               928
Ubisoft                          918
Konami Digital Entertainment     823
                                ... 
Detn8 Games                        1
Pow                                1
Navarre Corp                       1
MediaQuest                         1
UIG Entertainment                  1
Name: count, Length: 576, dtype: int64

In [7]:
data['Global_Sales'].describe() #Global_Sales

data.to_csv('mycsvfile.csv',index=False)

**Dashboard key metrics:**

*     Video game sales performance during release year
*     Total sales count and percentage for each region
*     Yearly sales from 1980 - 2020
*     Sales for top 5 platforms, genres, and publishers
*     Filters for year, platform, genre, and publisher


# TOP 5 Platforms Sales in each region?

In [8]:
query1 = psql.sqldf("""
                    SELECT Platform,
                    SUM(JP_Sales) AS JPN_Sales,
                    SUM(EU_Sales) AS EU_Sales,
                    SUM(NA_Sales) AS NA_Sales,
                    SUM(Global_Sales) AS Global_Sales 
                    FROM data
                    GROUP BY Platform
                    ORDER BY SUM(Global_Sales) DESC
                    LIMIT 5
                    
                    """)

query1

,Platform,JPN_Sales,EU_Sales,NA_Sales,Global_Sales
0,PS2,137.54,332.63,572.92,1233.46
1,X360,12.30,278.01,594.33,969.61
2,PS3,79.21,340.47,388.90,949.35
3,Wii,68.28,264.35,497.37,909.81
4,DS,175.02,194.07,388.56,818.96


# TOP 5 Video Games in each region?

In [9]:
query2 = psql.sqldf("""
                    SELECT Name, JP_Sales, EU_Sales, NA_Sales, Global_Sales
                    FROM data
                    GROUP BY Name
                    ORDER BY Global_Sales DESC
                    LIMIT 5
                    
                    """)
query2

,Name,JP_Sales,EU_Sales,NA_Sales,Global_Sales
0,Wii Sports,3.77,29.02,41.49,82.74
1,Super Mario Bros.,6.81,3.58,29.08,40.24
2,Mario Kart Wii,3.79,12.88,15.85,35.82
3,Wii Sports Resort,3.28,11.01,15.75,33.00
4,Pokemon Red/Pokemon Blue,10.22,8.89,11.27,31.37


# TOP 5 Video Games Publishers in each region?

In [10]:
query3 = psql.sqldf("""
                    SELECT Publisher,
                    SUM(JP_Sales) AS JPN_Sales,
                    SUM(EU_Sales) AS EU_Sales,
                    SUM(NA_Sales) AS NA_Sales,
                    SUM(Global_Sales) AS Global_Sales 
                    FROM data
                    GROUP BY Publisher
                    ORDER BY SUM(Global_Sales) DESC
                    LIMIT 5
                
                    
                    """)
query3

,Publisher,JPN_Sales,EU_Sales,NA_Sales,Global_Sales
0,Nintendo,454.99,418.30,815.75,1784.43
1,Electronic Arts,13.98,367.38,584.22,1093.39
2,Activision,6.54,213.72,426.01,721.41
3,Sony Computer Entertainment,74.10,187.55,265.22,607.28
4,Ubisoft,7.33,163.03,252.81,473.54


# Regional and Global Sales for Each Genre ?

In [11]:
query4 = psql.sqldf("""
                    SELECT Genre,
                    SUM(JP_Sales) AS JPN_Sales,
                    SUM(EU_Sales) AS EU_Sales,
                    SUM(NA_Sales) AS NA_Sales,
                    SUM(Other_Sales) AS Other_Sales,
                    SUM(Global_Sales) AS Global_Sales 
                    FROM data
                    GROUP BY Genre
                    ORDER BY SUM(Global_Sales) DESC
                
                    
                    """)
query4

,Genre,JPN_Sales,EU_Sales,NA_Sales,Other_Sales,Global_Sales
0,Action,158.66,516.48,861.80,184.92,1722.88
1,Sports,134.76,371.34,670.09,132.65,1309.24
2,Shooter,38.18,310.45,575.16,101.90,1026.20
3,Role-Playing,350.29,187.58,326.50,59.38,923.84
4,Platform,130.65,200.67,445.99,51.51,829.15
5,Misc,106.67,213.82,402.48,74.02,797.62
6,Racing,56.61,236.32,356.93,76.68,726.77
7,Fighting,87.15,100.00,220.74,36.19,444.05
8,Simulation,63.54,113.20,181.78,31.36,390.16
9,Puzzle,56.68,50.53,122.01,12.47,242.22


# Growth Of Action Games Over the Years

In [12]:
query5 = psql.sqldf("""
                    WITH LaggedSales AS (
                        SELECT Year, 
                        SUM(Global_Sales) AS Global_Sales,
                        LAG(SUM(Global_Sales),1) OVER (
                            ORDER BY Year
                        ) AS Last_global_Sales
                        FROM data
                        WHERE Genre = 'Shooter'
                        GROUP BY Year
                    )
                    
                    SELECT
                        Year,
                        Global_Sales,
                        ROUND(((Global_Sales - Last_global_Sales) * 100.0 / NULLIF(Last_global_Sales, 0)), 2) || '%' AS Growth_Percentage
                    FROM
                        LaggedSales
                    ORDER BY
                        Year;
                    
                    """)
query5

,Year,Global_Sales,Growth_Percentage
0,1980.0,7.07,None
1,1981.0,10.04,42.01%
2,1982.0,3.79,-62.25%
3,1983.0,0.48,-87.34%
4,1984.0,31.10,6379.17%
5,1985.0,1.00,-96.78%
6,1986.0,3.89,289.0%
7,1987.0,0.71,-81.75%
8,1988.0,0.51,-28.17%
9,1989.0,1.20,135.29%
